## Working with Key/Value Pairs (ch 4)

Spark provides special operations on RDDs containing key/value pairs. 

These RDDs are called *pair RDDs*. Pair RDDs are a useful building block in many programs, as they expose operations that allow you to act on each key in parallel or regroup data across the network. For example, pair RDDs have a **reduceByKey()** method that can aggregate data separately for each key, and a **join()** method that can merge two RDDs together by grouping elements with the same key. 

It is common to extract fields from an RDD (representing, for instance, an event time, customer ID, or other identifier) and use those fields as keys in pair RDD operations.

In [0]:
lines = sc.textFile('/FileStore/tables/Dante_Inferno.txt')
lines = lines.map(lambda s: s.replace('\x92',''))

In [0]:
lines.count()

Out[5]: 6496

create a set of `<key,value>` pairs where the key is the length of the line

In [0]:
linesWithLength = lines.map(lambda x: (len(x), x))
linesWithLength

Out[6]: PythonRDD[183] at RDD at PythonRDD.scala:58

In [0]:
linesWithLength.take(10)

Out[7]: [(20, ' LA DIVINA COMMEDIA'),
 (20, ' di Dante Alighieri'),
 (0, ''),
 (0, ''),
 (0, ''),
 (0, ''),
 (0, ''),
 (9, ' INFERNO'),
 (0, ''),
 (0, '')]

In [0]:
linesWithLength.sortByKey(ascending=False).take(10)

Out[8]: [(50, ' Li occhi mi sciolse e disse: «Or drizza il nerbo'),
 (49, ' che è, che i ben del mondo ha sì tra branche?».'),
 (49, ' quell altro foco? e chi son quei che l fenno?».'),
 (49, ' con li altri, innanzi a li altri aprì la canna,'),
 (49, ' da bocca il freddo, e da li occhi il cor tristo'),
 (48, ' se l ciel li addolcia o lo nferno li attosca».'),
 (48, ' che mostri in cielo, in terra e nel mal mondo,'),
 (48, ' Io cominciai: «O frati, i vostri mali . . . »;'),
 (48, ' se non fosse il gran prete, a cui mal prenda!,'),
 (48, ' che l ventre innanzi a li occhi sì tassiepa!».')]

## exercise: compute the average length of the lines using the keys -- without built in stats functions

In [0]:
sumLength = linesWithLength.keys().reduce(lambda x,y: x+y)

In [0]:
sumLength

Out[10]: 178352

In [0]:
avgLength = sumLength / float(linesWithLength.count())
avgLength

Out[11]: 27.45566502463054

select lines that are shorter than the average length

In [0]:
shortLines = linesWithLength.filter(lambda pair: pair[0] < avgLength)

In [0]:
linesWithLength.count(),shortLines.count()

Out[13]: (6496, 1776)

The canonical MapReduce **word count example**: computes <word, count> for each word in the document

Split each line into words

In [0]:
words = lines.flatMap(lambda l: l.split(" "))

filter out noise (optional step)

In [0]:
filteredWords = words.filter(lambda x:  x != ''  and not x.startswith('\\') and len(x) > 5)

count words

In [0]:
wordCount = filteredWords.map(lambda x: (x,1)).reduceByKey(lambda x, y: x+y)

In [0]:
wordCount.take(10)

Out[17]: [('DIVINA', 1),
 ('Alighieri', 1),
 ('Inferno', 34),
 ('cammin', 2),
 ('nostra', 14),
 ('ritrovai', 1),
 ('diritta', 1),
 ('morte;', 2),
 ('trattar', 1),
 ('trovai,', 1)]

#### Some simple analytics on the wordCount RDD: list the top-10 most frequent words

note that the pairs are not sorted by key, so we need to flip the pairs first, creating frequencyWordPairs

In [0]:
wordCount.sortBy(lambda x: x[1], ascending=False).take(5)

Out[18]: [('quando', 80),
 ('quella', 68),
 ('disse:', 55),
 ('laltro', 44),
 ('perché', 43)]

In [0]:
frequencyWordPairs = wordCount.map(lambda x: (x[1],x[0]))
frequencyWordPairs.take(10)

Out[19]: [(1, 'DIVINA'),
 (1, 'Alighieri'),
 (34, 'Inferno'),
 (2, 'cammin'),
 (14, 'nostra'),
 (1, 'ritrovai'),
 (1, 'diritta'),
 (2, 'morte;'),
 (1, 'trattar'),
 (1, 'trovai,')]

In [0]:
frequencyWordPairs.sortByKey(ascending=False).take(10)

Out[20]: [(80, 'quando'),
 (68, 'quella'),
 (55, 'disse:'),
 (44, 'laltro'),
 (43, 'perché'),
 (39, 'questa'),
 (38, 'questo'),
 (34, 'maestro'),
 (34, 'Inferno'),
 (31, 'dentro')]

### exercise: group words into sets, words in each set have the same frequency: <frequency, [words]>

we introduce a new grouping function:  **groupByKey()**

In [0]:
sameFrequencyGroups = frequencyWordPairs.groupByKey()
sameFrequencyGroups

Out[21]: PythonRDD[220] at RDD at PythonRDD.scala:58

In [0]:
sameFrequencyGroups.sortByKey(ascending=False)

Out[22]: PythonRDD[227] at RDD at PythonRDD.scala:58

In [0]:
sameFrequencyGroups.take(3)

Out[23]: [(26, <pyspark.resultiterable.ResultIterable at 0x7fcf48057df0>),
 (10, <pyspark.resultiterable.ResultIterable at 0x7fcf48057c70>),
 (2, <pyspark.resultiterable.ResultIterable at 0x7fcf48057850>)]

suppose we are only interested in groups with more than one element

In [0]:
nonSingletonGroups = sameFrequencyGroups.filter(lambda x: len(x[1])>1)

In [0]:
for (f,l) in nonSingletonGroups.take(7):
    print("words with frequency ",f)
    for w in l:
        print(w)

words with frequency 2
traccia
martiro
sapere
medesma
venimmo;
sgridò:
allor,
lusinghe
graffia
grandi
seguaci
tomba,
sapïenza,
livida
tondo.
stava.
strema
segnore,
volgemmo
ritto,
«Dilli
rispuosi
sospirando
fossi,
folle,
punito;
chesser
labbia
rimontò
discese.
quarto
quinto
scoperto
bagnava
mirabilmente
piangea,
convenne
avvolti,
Italia
grembo
vergine
pantano,
pantan
morte;
sorte.
inganno
menzogna
ricordar,
parlava,
fessura
pianti
Malebranche,
gridar:
raffi,
attuffare
chalcun
fatta,
sarresta,
«Nessun
voluto
cammin
landare
Libicocco
castella,
campar
peccatori;
ncontra
sentir
coperto,
disse;
piglio
domandò
barattier
lingue
«Fatti
malvagio
volete
ricominciò
troppo,
altri,
potero
falcon
crucciato
lasciammo
pensava
crudeli
roccia,
volger
figlio,
giunti
occhi,
giunti,
fummo,
angeli
adunque:
assai,
vanno,
campagna
scheggia
corta,
lascia,
fornito
meglio
fosso,
cinghio
lopera
lottava
mostrò
sperar
angoscia
piovvi
Pistoia
dicesse
Pistoia,
Maremma
quante
groppa
labbia.
cessar
lancia
bianco
venendo
febbre
troppa
novelle
settima
novità
fuggirsi
chiusi,
cittadini
tempo,
drizzo
«Dentro
fuochi
sopra,
porta».
faville
venuta
meritai
esperto
bagna.
Ercule
quarta
piacque,
forame
dandole
restare
guerra;
Romagna
lasciai.
bianco,
tirannia
stato,
credesse
guerir
peggio,
Padre,
veggia,
dunque
altro,
favella.
rimasi
prova,
certo,
dicea:
volge.
pianga
maestro:
vïolenta
lamenti
sinistra;
ministra
malizia,
tutti,
oscura
volontier
daltro
sconcia
Capocchio,
falsai
divenne
moglie
artigli,
nsieme
bestie,
«Quell
«quando
oscuro,
Quella
larsura
agogna,
tuttavia,
trestizia
corno,
seguitando,
giganti
puote,
feroce
volto».
tremoto
allotta,
freddura
render
grazia
chinato,
vederlo
Lucifero
luniverso,
gigante
bassi,
dicere
freddo
sovente
vinse.
destino
fretta».
dimandi
converrà
favelle,
tempie
teschio
convegno,
fiorentino
lorribile
piangëa,
raggio
misere
diedi,
dolor,
fredda
apersi;
costume
fortezza
divenni
dingegno,
vegnon
dolenti.
credetti
piede:
«Prima
riveder
verace
giunto,
piaggia
lerta,
acquista,
gridai
discerno
lascerò
«Poeta,
quale,
dirotti
sospesi,
chiamò
diserta
centro
dimando
lantica
danno,
onesto,
ragionato
intrai
venuti
cinta,
tegnon
avrebber
Rispuose:
gente,
riviera
vuole,
dimandare».
nocchier
similemente
avanti
«Figliuol
mossi,
primo,
secondo».
terzo,
convene
compagni,
caccia
discesi
Sempre
vicenda
conforta
biasmo
venite
martìri
eravamo
pietade
mente,
tormenti
intorno,
cerchio,
crudele
bramose
messo,
saltra
rispuosi:
strada,
nemico.
labbia,
scendemmo
ciascun,
punto,
dimostra
«Tutti
coperchio
chiuso,
creature
difension
fangose
tratto
negato,
forse,
segnor,
question
passava
strada
condizion
guardia
manifesto
Dintorno
piena.
aspetto,
«Selli
tormenta
regge,
pregai
lantico
distinto
prossimo
spregiando
offende
mezzo,
«Siete
Rinier
credette
glorïoso
disdegnoso
cespuglio
fronde
cadere
divenimmo
vedrai,
solver
rivolse,
sponda
vaneggia
facean
rinova
guardai
presta
ritornar
vòlto.
mattino,
gravezza
volontieri
perder
fioco.
patrïa
ritorni
lascia
caccerà
villa,
nvidia
guida,
strida,
seconda
vederai
contenti
impera
possente,
parente,
intendi
pensando,
viltade
impresa
favella:
impedito
allora,
spezie
talento.
guardi
dentro,
ncendio
fidandomi
chudito
volse:
ardire
disposto
tornato
Dinanzi
etterne,
perduto
sospiri,
stelle,
lingue,
tinta,
coloro
profondo
gloria
morte,
intesi
ignudi
lagrime,
trapassar
morti».
verrai
Quinci
chattende
disio.
riscossi
conoscer
«Langoscia
labisso
martìri,
passavam
selva,
laltissimo
cenno,
Venimmo
Questo
giugnemmo
gravi,
Cesare
pieno,
primaio
secondo,
quantunque
lasciando
offizio,
piena,
venir,
rotta,
parlar,
sospetto.
leggemmo
questi,
chiuse
vermo,
vedessi
cittadin
superbia,
suono.
Farinata
Iacopo
tromba,
figura,
lenti,
sentenza,
Michele
cherci
chiaro
fortuna,
quanta
conduce
necessità
stella
inteso,
offeso.
posson
chassai
fossimo
risponde
saetta
ascolta
altrui.
vegno,
piangere
duolo,
figliuolo,
sappressa
chentro
stette,
stette
avversari
scritta
menare
chaltra
intrar
strani.
avversi
movemmo
bagna,
chiede
seguaci,
secreto
spalle.
dimanda
pinse

**Exercise**: create a set of words for each frequency value, using **combineByKey()** in combination with the frequencyWordPairs RDD

### Joins

Example: (id, age) join (id, salary) using id as key:

In [0]:
idAge = sc.parallelize([(1,20), (2,25), (3,40), (4,45), (3,50)])
idIncome = sc.parallelize([(1,10000), (2,30000), (3,40000)])

1. full join. note that this leaves out id=4  (why??)

In [0]:
idAge.join(idIncome).collect()

Out[27]: [(1, (20, 10000)), (2, (25, 30000)), (3, (40, 40000)), (3, (50, 40000))]

outer join -- includes id = 4

In [0]:
idAge.leftOuterJoin(idIncome).collect()

Out[28]: [(1, (20, 10000)),
 (2, (25, 30000)),
 (3, (40, 40000)),
 (3, (50, 40000)),
 (4, (45, None))]